#Mount Drive

In [0]:
from google.colab import drive
drive.mount("./gdrive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./gdrive/


#Loading Data

In [0]:
import torch
import torchtext
import spacy

nlp=spacy.load("en")

def tokenizer(sentence):
  return [token.text for token in nlp.tokenizer(sentence)]

In [0]:
from torchtext.data import Field

TEXT=Field(sequential=True,lower=True,tokenize=tokenizer,init_token="<start>",eos_token="<end>",batch_first=True)
LABEL=Field(sequential=False,batch_first=True,unk_token=None)

In [0]:
from torchtext.data import TabularDataset

root="./gdrive/My Drive/Amazon Review Dataset"
fields={"label":("label",LABEL),"sentence":("sentence",TEXT)}

train,val,test=TabularDataset.splits(path=root,train="train.json", validation="valid.json", test="test.json",format="json",fields=fields)

In [0]:
TEXT.build_vocab(train, vectors="glove.6B.300d")               # can specify vocab_size
LABEL.build_vocab(train)

.vector_cache/glove.6B.zip: 862MB [00:34, 25.2MB/s]                           
100%|█████████▉| 399111/400000 [00:38<00:00, 10497.33it/s]

In [0]:
print(len(TEXT.vocab))
print(LABEL.vocab.stoi)
print(LABEL.vocab.freqs)

58714
defaultdict(<function _default_unk_index at 0x7f21aa93d8c8>, {1: 0, 0: 1})
Counter({1: 10056, 0: 9908})


In [0]:
print(TEXT.vocab.stoi["<end>"])
print(TEXT.vocab.stoi["<pad>"])
print(TEXT.vocab.stoi["n't"])

print(TEXT.vocab.itos[55])


3
1
30
has


In [0]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 125253), ('.', 121329), (',', 96666), ('and', 66508), ('i', 62321), ('to', 60863), ('a', 59332), ('it', 51535), ('of', 46838), ('is', 41696), (' ', 41276), ('this', 34363), ('in', 30303), ('that', 27083), ('for', 25881), ('you', 20498), ('with', 19470), ('was', 18007), ("'s", 17939), ('on', 17860)]


##creating data-iterators

In [0]:
batch_size=64

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

#device=torch.device("cpu")

train_iter, val_iter, test_iter= torchtext.data.BucketIterator.splits( (train,val,test), sort_key=lambda x: len(x.sentence), batch_size=batch_size, device=device)         


In [0]:
print(device)

cuda


In [0]:
for batch in train_iter:
  print(batch.label)
  print(batch.label.shape)
  print(batch.sentence)
  print(batch.sentence.shape)
  break

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
        1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
torch.Size([64])
tensor([[   2,  601,   15,  ...,    1,    1,    1],
        [   2,    8,  129,  ...,    1,    1,    1],
        [   2,   15, 1926,  ...,    1,    1,    1],
        ...,
        [   2,    8,  152,  ...,    1,    1,    1],
        [   2, 9497,   84,  ...,    1,    1,    1],
        [   2,    8,  242,  ...,    1,    1,    1]], device='cuda:0')
torch.Size([64, 568])


In [0]:
len(train_iter)

312

## loading pre-trained weight vectors

In [0]:
weight_matrix=TEXT.vocab.vectors
vocab_size, emb_dim= weight_matrix.shape

In [0]:
print(weight_matrix[58713,:])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

#Model

In [0]:
from torch import nn

class Classifier(nn.Module):
  
  def __init__(self , vocab_size, emb_dim, hidden_dim):
    super().__init__()
    
    self.embedding= nn.Embedding.from_pretrained(weight_matrix)
    self.gru = nn.GRU(emb_dim, hidden_dim, num_layers=2, batch_first=True, bidirectional=True)
    self.ff1 = nn.Linear(4 * hidden_dim, 128)
    self.ff2 = nn.Linear(128,1)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    
  def forward(self, x):
    
    emb = self.embedding(x)
    lstm_out, h = self.gru(emb)
    h=h.permute(1,0,2)
    f1_out = self.relu(self.ff1(h.reshape(h.shape[0],-1)))
    output = self.sigmoid(self.ff2(f1_out))
    
    
    return output.squeeze(-1)

In [0]:
epochs=40
hidden_dim=128

model=Classifier(vocab_size, emb_dim, hidden_dim)
model.to(device)



Classifier(
  (embedding): Embedding(58714, 300)
  (gru): GRU(300, 128, num_layers=2, batch_first=True, bidirectional=True)
  (ff1): Linear(in_features=512, out_features=128, bias=True)
  (ff2): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()
criterion.to(device)

BCELoss()

In [0]:
def binary_accuracy(pred, labels):
  
  rounded_pred=torch.round(pred)
  equal= (rounded_pred==labels.view(*rounded_pred.shape)).float()
  accuracy= equal.sum()/len(equal)
  
  return accuracy

#Training

In [0]:
import math

min_val_loss=10000000000000

training_acc = []
training_losses = []
validation_acc = []
validation_losses = []


for epoch in range(1,epochs+1):
  
  train_acc = 0
  train_loss = 0
  val_acc = 0
  val_loss = 0
  
  print("Epoch: ",epoch)
  
  model.train()
  
  t=0
  step=25/len(train_iter)
  for batch in train_iter:
    t+=1
    
    
    batch.label=batch.label.type(torch.FloatTensor)
      
    batch.sentence=batch.sentence.to(device)
    batch.label=batch.label.to(device)
    
    
    optimizer.zero_grad()
    
    predictions = model(batch.sentence)
    
    loss = criterion(predictions,batch.label)
    acc=binary_accuracy(predictions,batch.label)
    
    loss.backward()
    optimizer.step()
    
    train_loss+=loss.item()
    train_acc+=acc.item()
    
    
    print('\r' + f'Training: '
                f"[{'=' * int((t) * step) + ' ' * (24 - int((t) * step))}]"
                f"({math.ceil((t) * 100 /len(train_iter))} %)",
                end='')
  
  
  train_acc/=len(train_iter)
  train_loss/=len(train_iter)
  print('\nTraining Loss: {:.6f} /t Training accuracy: {: .6f}'.format( train_loss,train_acc))
  training_acc.append(train_acc)
  training_losses.append(train_loss)
  
  
  with torch.no_grad():
    
    model.eval()
    t=0
    step=25/len(val_iter)
    for val_batch in val_iter:
      t+=1
      
      val_batch.label=val_batch.label.type(torch.FloatTensor)
      
      val_batch.sentence=val_batch.sentence.to(device)
      val_batch.label=val_batch.label.to(device)
      
      predictions = model(val_batch.sentence)
      loss = criterion(predictions,val_batch.label)
      acc=binary_accuracy(predictions,val_batch.label)
      
      
      val_loss+=loss.item()
      val_acc+=acc.item()
      
      
      
      print('\r' + f'Validation: '
                f"[{'=' * int((t) * step) + ' ' * (24 - int((t) * step))}]"
                f"({math.ceil((t) * 100 /len(val_iter))} %)",
                end='')
      
    val_acc/=len(val_iter)
    val_loss/=len(val_iter)
    print('\nValidation Loss: {:.6f} /t Validation accuracy: {: .6f}'.format( val_loss,val_acc))
    validation_acc.append(val_acc)
    validation_losses.append(val_loss)
    
    if val_loss< min_val_loss:
      min_val_loss= val_loss
      
      print("Validation loss reduced to {:.6f}, saving model............".format(val_loss))
      torch.save(model.state_dict(), 'text-model.epoch='+str(epoch)+'.pt')
      print()
  

Epoch:  1
Training: [=                       ](5 %)

100%|█████████▉| 399111/400000 [00:50<00:00, 10497.33it/s]

Training: [=========================](100 %)
Training Loss: 0.528513 /t Training accuracy:  0.733480
Validation: [=========================](100 %)
Validation Loss: 0.419511 /t Validation accuracy:  0.820268
Validation loss reduced to 0.419511, saving model............

Epoch:  2
Training: [=========================](100 %)
Training Loss: 0.367409 /t Training accuracy:  0.841877
Validation: [=========================](100 %)
Validation Loss: 0.344047 /t Validation accuracy:  0.867411
Validation loss reduced to 0.344047, saving model............

Epoch:  3
Training: [=========================](100 %)
Training Loss: 0.290215 /t Training accuracy:  0.880335
Validation: [=========================](100 %)
Validation Loss: 0.299775 /t Validation accuracy:  0.872143
Validation loss reduced to 0.299775, saving model............

Epoch:  4
Training: [=========================](100 %)
Training Loss: 0.237038 /t Training accuracy:  0.905746
Validation: [=========================](100 %)
Validatio

KeyboardInterrupt: ignored

In [0]:
import shutil

for i in range(1,5):
  shutil.copy("./text-model.epoch="+str(i)+".pt","./gdrive/My Drive/Amazon Review Dataset/GRU/text-model.epoch="+str(i)+".pt")
  

#Testing

In [0]:
state_dict=torch.load("./gdrive/My Drive/Amazon Review Dataset/GRU/text-model.epoch=4.pt")
model.load_state_dict(state_dict)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
model.eval()

Classifier(
  (embedding): Embedding(58714, 300)
  (gru): GRU(300, 128, num_layers=2, batch_first=True, bidirectional=True)
  (ff1): Linear(in_features=512, out_features=128, bias=True)
  (ff2): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [0]:
with torch.no_grad():
    test_acc =0
    test_loss=0
    t=0
    step=25/len(test_iter)
    for test_batch in test_iter:
      t+=1
      
      test_batch.label=test_batch.label.type(torch.FloatTensor)
      
      test_batch.sentence=test_batch.sentence.to(device)
      test_batch.label=test_batch.label.to(device)
      
      predictions = model(test_batch.sentence)
      loss = criterion(predictions,test_batch.label)
      acc=binary_accuracy(predictions,test_batch.label)
      
      
      test_loss+=loss.item()
      test_acc+=acc.item()
      
      
      
      print('\r' + f'Testing: '
                f"[{'=' * int((t) * step) + ' ' * (24 - int((t) * step))}]"
                f"({math.ceil((t) * 100 /len(test_iter))} %)",
                end='')
      
    test_acc/=len(test_iter)
    test_loss/=len(test_iter)
    print('\nTest Loss: {:.6f} /t Test accuracy: {: .6f}'.format( test_loss,test_acc))
    

Testing: [=========================](100 %)
Test Loss: 0.280789 /t Test accuracy:  0.885121
